Team Name: Pathfinder \
Team Member: Vignesh S S\
Problem Statememnt: AI-Based Computer Vision for Healthcare Hackathon\
Dataset Link: https://drive.google.com/drive/u/1/folders/1UVvAea975DxWIYFSULqmqVMKLquBOs1Y \


In [ ]:
#!pip install gdown
#!gdown --folder 1UVvAea975DxWIYFSULqmqVMKLquBOs1Y  --remaining-ok

In [1]:
# Install the library for segmentation models
!pip install -U torchmetrics albumentations segmentation-models-pytorch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.2/983.2 kB 24.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.8/154.8 kB 17.2 MB/s eta 0:00:00


In [8]:
from google.colab import drive
import os

# Mount your Google Drive at /content/drive
drive.mount('/content/drive')

# --- Configuration ---
# IMPORTANT: Change this to the exact name of your zip file in Google Drive
ZIP_FILE_NAME = "/Hackathon_Dataset_20251025T071740Z.zip"
ZIP_PATH_ON_DRIVE = f"/content/{ZIP_FILE_NAME}"
UNZIP_DESTINATION = "/content/"  # Unzips to the fast local /content/ disk

# --- Unzip Logic ---
if os.path.exists(ZIP_PATH_ON_DRIVE):
    print(f"Found zip file at: {ZIP_PATH_ON_DRIVE}")
    print("Unzipping to local disk... This may take a minute.")
    # -q (quiet) suppresses the long list of all 200+ files
    !unzip -q "{ZIP_PATH_ON_DRIVE}" -d "{UNZIP_DESTINATION}"
    print("Unzipping complete!")

    # Verify the expected folder is now present
    print("Contents of /content/ (your unzipped data):")
    !ls "/content/"
else:
    print(f"ERROR: Zip file not found at: {ZIP_PATH_ON_DRIVE}")
    print("Please check the file name and its location in your Google Drive.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Found zip file at: /content//Hackathon_Dataset_20251025T071740Z.zip
Unzipping to local disk... This may take a minute.
Unzipping complete!
Contents of /content/ (your unzipped data):
 drive		      Hackathon_Dataset_20251025T071740Z.zip
'Hackathon Dataset'   sample_data


In [9]:
# --- 1. Imports ---
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
# Import the learning rate scheduler
from torch.optim.lr_scheduler import ReduceLROnPlateau
import cv2  # OpenCV for image processing
import numpy as np
import os
import glob  # For finding files
from tqdm.notebook import tqdm  # For progress bars
import albumentations as A
import segmentation_models_pytorch as smp
import torchmetrics  # For IoU/Jaccard metric

In [13]:
# --- 2. Helper Function: Letterbox Padding ---

def letterbox_image(image, mask, target_size=(384, 384)):
    """
    Resizes and pads an image and its mask to a target square size
    while maintaining the original aspect ratio.

    Args:
        image (np.array): The input image (H, W).
        mask (np.array): The corresponding segmentation mask (H, W).
        target_size (tuple): The desired output size (height, width).

    Returns:
        tuple: A tuple (padded_image, padded_mask).
    """
    h, w = image.shape[:2]  # Original height, width

    # Calculate scale factor to fit the longest side
    scale = min(target_size[0] / h, target_size[1] / w)
    new_w, new_h = int(w * scale), int(h * scale)

    # Use INTER_LINEAR for resizing the image (good quality)
    resized_image = cv2.resize(
        image, (new_w, new_h), interpolation=cv2.INTER_LINEAR
    )

    # Use INTER_NEAREST for the mask. This is CRITICAL.
    # It prevents creating new class values (e.g., 0.5) during resize.
    resized_mask = cv2.resize(
        mask, (new_w, new_h), interpolation=cv2.INTER_NEAREST
    )

    # Calculate padding for top/bottom and left/right
    top = (target_size[0] - new_h) // 2
    bottom = target_size[0] - new_h - top
    left = (target_size[1] - new_w) // 2
    right = target_size[1] - new_w - left

    # Pad the image with black (value=0)
    padded_image = cv2.copyMakeBorder(
        resized_image, top, bottom, left, right,
        borderType=cv2.BORDER_CONSTANT, value=0
    )

    # Pad the mask with the background class (value=0)
    padded_mask = cv2.copyMakeBorder(
        resized_mask, top, bottom, left, right,
        borderType=cv2.BORDER_CONSTANT, value=0
    )
    return padded_image, padded_mask


# --- 3. Custom PyTorch Dataset Class ---

class FetalHeartDataset(Dataset):
    """
    Custom Dataset for loading fetal heart ultrasound images and
    dynamically creating segmentation masks from ellipse .txt files.
    """
    def __init__(self, image_paths, label_paths, target_size=(384, 384),
                 augmentations=None):
        """
        Args:
            image_paths (list[str]): List of file paths to the images.
            label_paths (list[str]): List of file paths to the .txt labels.
            target_size (tuple): The target size for letterbox padding.
            augmentations (A.Compose, optional): Albumentations pipeline.
        """
        self.image_paths = image_paths
        self.label_paths = label_paths
        self.target_size = target_size
        self.augmentations = augmentations
        # Map class names to integer values for the mask
        self.class_map = {"cardiac": 1, "thorax": 2}  # 0 is background

    def __len__(self):
        """Returns the total number of samples in the dataset."""
        return len(self.image_paths)

    def create_mask_from_txt(self, label_path, img_shape):
        """Generates a 3-class mask from ellipse coordinates in a .txt file."""
        # Start with a black mask (all zeros = background class)
        mask = np.zeros(img_shape, dtype=np.uint8)
        try:
            with open(label_path, 'r') as f:
                for line in f:
                    parts = line.strip().split()
                    if len(parts) != 6:
                        continue  # Skip malformed lines

                    # Parse data from the line
                    center_x = float(parts[0])
                    center_y = float(parts[1])
                    # cv2.ellipse expects radius (half of the axis)
                    major_axis = float(parts[2]) / 2
                    minor_axis = float(parts[3]) / 2
                    angle = float(parts[4])
                    class_name = parts[5]

                    class_value = self.class_map.get(class_name)
                    if class_value is not None:
                        # Draw the filled ellipse on the mask
                        cv2.ellipse(
                            img=mask,
                            center=(int(center_x), int(center_y)),
                            axes=(int(major_axis), int(minor_axis)),
                            angle=angle,
                            startAngle=0,
                            endAngle=360,
                            color=class_value,
                            thickness=-1  # -1 means filled
                        )
        except Exception as e:
            print(f"Error processing mask file {label_path}: {e}")
        return mask

    def __getitem__(self, idx):
        """
        Fetches one sample (image, mask) from the dataset, processes it,
        and returns it as a PyTorch Tensor.
        """
        # Load image as grayscale
        img_path = self.image_paths[idx]
        image = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        if image is None:
            raise IOError(f"Could not read image: {img_path}")

        # Create corresponding mask
        label_path = self.label_paths[idx]
        mask = self.create_mask_from_txt(label_path, image.shape[:2])

        # Apply letterbox padding to maintain aspect ratio
        image, mask = letterbox_image(image, mask, self.target_size)

        # Apply augmentations (if any are provided)
        if self.augmentations:
            augmented = self.augmentations(image=image, mask=mask)
            image = augmented['image']
            mask = augmented['mask']

        # Convert to PyTorch tensors
        # Add channel dimension: (H, W) -> (1, H, W)
        image = torch.from_numpy(image).float().unsqueeze(0)
        # Mask should be LongTensor, no channel dim: (H, W)
        mask = torch.from_numpy(mask).long()

        # Normalize image to [0, 1]
        image = image / 255.0

        return image, mask


# --- 4. Define Augmentation Pipelines ---

# Augmentations for the training set to prevent overfitting
train_augs = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.ShiftScaleRotate(
        shift_limit=0.1, scale_limit=0.1, rotate_limit=30, p=0.7,
        border_mode=cv2.BORDER_CONSTANT, value=0
    ),
    A.ElasticTransform(
        p=0.3, alpha=120, sigma=120 * 0.05, alpha_affine=120 * 0.03,
        border_mode=cv2.BORDER_CONSTANT, value=0
    ),
    A.RandomBrightnessContrast(p=0.5),
    A.GaussNoise(p=0.3),
    A.MotionBlur(p=0.3),
])

# No augmentations for the validation set, just padding (handled by Dataset)
val_augs = None

# --- 5. Set Up Paths and DataLoaders (CORRECTED) ---

# Base path where the dataset was unzipped
BASE_DATA_PATH = "/content/Hackathon Dataset/"

# --- Define dataset-specific paths ---
TRAIN_DIR = os.path.join(BASE_DATA_PATH, "Images/training")
VAL_DIR = os.path.join(BASE_DATA_PATH, "Images/validation")

# --- Find all image and label files ---

# Get Training paths
# 1. Find all image files
train_img_paths = sorted(glob.glob(os.path.join(TRAIN_DIR, "images", "*.png")))
# 2. Create label paths *based on* the image paths
train_lbl_paths = [
    p.replace("/images/", "/annfiles_ellipse/")  # Swap folder name
     .replace(".png", ".txt")                   # Swap extension
    for p in train_img_paths
]

# Get Validation paths
# 1. Find all image files
val_img_paths = sorted(glob.glob(os.path.join(VAL_DIR, "images", "*.png")))
# 2. Create label paths *based on* the image paths
val_lbl_paths = [
    p.replace("/images/", "/annfiles_ellipse/")  # Swap folder name
     .replace(".png", ".txt")                   # Swap extension
    for p in val_img_paths
]

# --- Verification Step ---
print(f"Found {len(train_img_paths)} training images.")
print(f"Created {len(train_lbl_paths)} matching training labels.")
print(f"Found {len(val_img_paths)} validation images.")
print(f"Created {len(val_lbl_paths)} matching validation labels.")

# This check is now much more robust
if len(train_img_paths) != len(train_lbl_paths) or len(train_img_paths) == 0:
    print("\n--- ERROR ---")
    print("Mismatch or zero files found. Check your paths and folder structure!")
else:
    print("\n--- Path Check (Good) ---")
    print("Image path [0]:", train_img_paths[0])
    print("Label path [0]:", train_lbl_paths[0])
    # You can add a check here to see if the file actually exists
    if not os.path.exists(train_lbl_paths[0]):
        print(f"WARNING: Label file {train_lbl_paths[0]} does not exist!")

# --- Create Dataset Instances ---
TARGET_IMG_SIZE = (384, 384)

train_dataset = FetalHeartDataset(
    train_img_paths, train_lbl_paths,
    target_size=TARGET_IMG_SIZE,
    augmentations=train_augs
)
val_dataset = FetalHeartDataset(
    val_img_paths, val_lbl_paths,
    target_size=TARGET_IMG_SIZE,
    augmentations=val_augs
)

# --- Create DataLoaders ---
BATCH_SIZE = 8
train_loader = DataLoader(
    train_dataset, batch_size=BATCH_SIZE, shuffle=True,
    num_workers=2, pin_memory=True
)
val_loader = DataLoader(
    val_dataset, batch_size=BATCH_SIZE, shuffle=False,
    num_workers=2, pin_memory=True
)

Found 198 training images.
Created 198 matching training labels.
Found 50 validation images.
Created 50 matching validation labels.

--- Path Check (Good) ---
Image path [0]: /content/Hackathon Dataset/Images/training/images/001.png
Label path [0]: /content/Hackathon Dataset/Images/training/annfiles_ellipse/001.txt


/tmp/ipython-input-3580307607.py:158: UserWarning: Argument(s) 'value' are not valid for transform ShiftScaleRotate
  A.ShiftScaleRotate(
/tmp/ipython-input-3580307607.py:162: UserWarning: Argument(s) 'alpha_affine, value' are not valid for transform ElasticTransform
  A.ElasticTransform(


In [14]:
# --- 6. Model, Loss, Optimizer & Metrics ---

# --- Hyperparameters ---
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
NUM_CLASSES = 3      # 0: background, 1: cardiac, 2: thorax
NUM_EPOCHS = 25      # Start with 20-25 epochs
LEARNING_RATE = 1e-4

# --- Model (U-Net) ---
model = smp.Unet(
    encoder_name="resnet34",      # Pre-trained backbone
    encoder_weights="imagenet",   # Use ImageNet weights (fine-tuning)
    in_channels=1,                # 1 channel for grayscale images
    classes=NUM_CLASSES,          # 3 output classes
).to(DEVICE)

# --- Loss Function ---
# DiceLoss is excellent for imbalanced segmentation tasks
loss_fn = smp.losses.DiceLoss(mode='multiclass', from_logits=True)

# --- Optimizer ---
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

# --- Metric (Jaccard/IoU) ---
# This is the primary metric for segmentation, now imported
jaccard = torchmetrics.JaccardIndex(
    task="multiclass", num_classes=NUM_CLASSES
).to(DEVICE)

# --- LR Scheduler ---
# Automatically reduces learning rate if validation loss stops improving
scheduler = ReduceLROnPlateau(
    optimizer,
    mode='min',      # Track the minimum of the validation loss
    factor=0.1,      # Reduce LR by a factor of 10
    patience=3,      # Wait 3 epochs with no improvement
    #verbose=True     # Print a message when LR is reduced
)

# --- Model Save Path ---
DRIVE_PATH = "/content/drive/My Drive/"
SAVE_PATH = os.path.join(
    DRIVE_PATH,
    "hackathon_submission/task_1_segmentation/Model Weights/best_model_hypothesis_1.pth"
)
# Ensure the save directory exists
os.makedirs(os.path.dirname(SAVE_PATH), exist_ok=True)

# Variable to track the best performance
best_val_iou = 0.0


# --- 7. Training Loop ---

print(f"--- Starting Training on {DEVICE} for {NUM_EPOCHS} epochs ---")

for epoch in range(NUM_EPOCHS):

    # --- Training Phase ---
    model.train()  # Set model to training mode (enables dropout, etc.)
    train_loss = 0.0

    # Use tqdm for a progress bar
    loop = tqdm(train_loader, desc=f"Epoch {epoch+1}/{NUM_EPOCHS} [Train]")
    for images, masks in loop:
        # Move data to the GPU
        images, masks = images.to(DEVICE), masks.to(DEVICE)

        # 1. Zero gradients from previous iteration
        optimizer.zero_grad()

        # 2. Forward pass: get model predictions
        outputs = model(images)  # Shape: [B, C, H, W]

        # 3. Calculate loss
        loss = loss_fn(outputs, masks)

        # 4. Backward pass: compute gradients
        loss.backward()

        # 5. Optimizer step: update model weights
        optimizer.step()

        train_loss += loss.item()
        # Update progress bar description with current batch loss
        loop.set_postfix(loss=loss.item())

    # --- Validation Phase ---
    model.eval()  # Set model to evaluation mode (disables dropout, etc.)
    val_loss = 0.0
    val_iou = 0.0

    loop_val = tqdm(val_loader, desc=f"Epoch {epoch+1}/{NUM_EPOCHS} [Val]")
    with torch.no_grad():  # Disable gradient calculation for efficiency
        for images, masks in loop_val:
            images, masks = images.to(DEVICE), masks.to(DEVICE)

            # Forward pass
            outputs = model(images)

            # Calculate loss
            loss = loss_fn(outputs, masks)
            val_loss += loss.item()

            # Calculate metric
            # Convert logits (raw outputs) to predicted class index (0, 1, or 2)
            preds = torch.argmax(outputs, dim=1)
            iou = jaccard(preds, masks)
            val_iou += iou.item()

            # Update progress bar
            loop_val.set_postfix(val_loss=loss.item(), val_iou=iou.item())

    # --- End of Epoch ---

    # Calculate average loss and IoU for the entire epoch
    avg_train_loss = train_loss / len(train_loader)
    avg_val_loss = val_loss / len(val_loader)
    avg_val_iou = val_iou / len(val_loader)

    print(f"Epoch {epoch+1} -> Train Loss: {avg_train_loss:.4f}, "
          f"Val Loss: {avg_val_loss:.4f}, Val IoU: {avg_val_iou:.4f}")

    # Step the scheduler based on validation loss
    scheduler.step(avg_val_loss)

    # --- Checkpoint Saving ---
    # Save the model if its validation IoU is the best so far
    if avg_val_iou > best_val_iou:
        print(f"Validation IoU improved ({best_val_iou:.4f} -> {avg_val_iou:.4f}). "
              f"Saving model to {SAVE_PATH}")
        best_val_iou = avg_val_iou
        # Save the model's state dictionary
        torch.save(model.state_dict(), SAVE_PATH)

print("--- Training Finished ---")
print(f"Best model (Val IoU: {best_val_iou:.4f}) saved to {SAVE_PATH}")

--- Starting Training on cuda for 25 epochs ---


Epoch 1/25 [Train]:   0%|          | 0/25 [00:00<?, ?it/s]

Error processing mask file /content/Hackathon Dataset/Images/training/annfiles_ellipse/047.txt: [Errno 2] No such file or directory: '/content/Hackathon Dataset/Images/training/annfiles_ellipse/047.txt'
Error processing mask file /content/Hackathon Dataset/Images/training/annfiles_ellipse/108.txt: [Errno 2] No such file or directory: '/content/Hackathon Dataset/Images/training/annfiles_ellipse/108.txt'
Error processing mask file /content/Hackathon Dataset/Images/training/annfiles_ellipse/189.txt: [Errno 2] No such file or directory: '/content/Hackathon Dataset/Images/training/annfiles_ellipse/189.txt'


Epoch 1/25 [Val]:   0%|          | 0/7 [00:00<?, ?it/s]

Error processing mask file /content/Hackathon Dataset/Images/validation/annfiles_ellipse/036.txt: [Errno 2] No such file or directory: '/content/Hackathon Dataset/Images/validation/annfiles_ellipse/036.txt'
Error processing mask file /content/Hackathon Dataset/Images/validation/annfiles_ellipse/043.txt: [Errno 2] No such file or directory: '/content/Hackathon Dataset/Images/validation/annfiles_ellipse/043.txt'
Epoch 1 -> Train Loss: 0.7822, Val Loss: 0.7413, Val IoU: 0.3236
Validation IoU improved (0.0000 -> 0.3236). Saving model to /content/drive/My Drive/hackathon_submission/task_1_segmentation/Model Weights/best_model_hypothesis_1.pth


...

Error processing mask file /content/Hackathon Dataset/Images/validation/annfiles_ellipse/036.txt: [Errno 2] No such file or directory: '/content/Hackathon Dataset/Images/validation/annfiles_ellipse/036.txt'
Error processing mask file /content/Hackathon Dataset/Images/validation/annfiles_ellipse/043.txt: [Errno 2] No such file or directory: '/content/Hackathon Dataset/Images/validation/annfiles_ellipse/043.txt'
Epoch 24 -> Train Loss: 0.3169, Val Loss: 0.3305, Val IoU: 0.6606


Epoch 25/25 [Train]:   0%|          | 0/25 [00:00<?, ?it/s]

Error processing mask file /content/Hackathon Dataset/Images/training/annfiles_ellipse/189.txt: [Errno 2] No such file or directory: '/content/Hackathon Dataset/Images/training/annfiles_ellipse/189.txt'
Error processing mask file /content/Hackathon Dataset/Images/training/annfiles_ellipse/108.txt: [Errno 2] No such file or directory: '/content/Hackathon Dataset/Images/training/annfiles_ellipse/108.txt'
Error processing mask file /content/Hackathon Dataset/Images/training/annfiles_ellipse/047.txt: [Errno 2] No such file or directory: '/content/Hackathon Dataset/Images/training/annfiles_ellipse/047.txt'


Epoch 25/25 [Val]:   0%|          | 0/7 [00:00<?, ?it/s]

Error processing mask file /content/Hackathon Dataset/Images/validation/annfiles_ellipse/043.txt: [Errno 2] No such file or directory: '/content/Hackathon Dataset/Images/validation/annfiles_ellipse/043.txt'
Error processing mask file /content/Hackathon Dataset/Images/validation/annfiles_ellipse/036.txt: [Errno 2] No such file or directory: '/content/Hackathon Dataset/Images/validation/annfiles_ellipse/036.txt'
Epoch 25 -> Train Loss: 0.2969, Val Loss: 0.3095, Val IoU: 0.6673
--- Training Finished ---
Best model (Val IoU: 0.6726) saved to /content/drive/My Drive/hackathon_submission/task_1_segmentation/Model Weights/best_model_hypothesis_1.pth
